# Data preparation

In [1]:
%matplotlib inline
import csv
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from datetime import date
import re
import matplotlib.pyplot as plt
import PandasHelper as pdh
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")


In [2]:
DATA_PATH = 'data/'
MAIN_FILE = DATA_PATH+'nielsen.csv'

In [3]:
df = pd.read_csv(MAIN_FILE, parse_dates=['first_timeframe'],nrows=1000)
df['date']=df['first_timeframe'].dt.date
df.head(2)

,site_name,first_timeframe,dwell_time_s,device_id,visitor,date
0,BF Karlsruhe Kaiserstr (1122),2014-12-31 23:00:30,15,bd5d8c2890622782d681c82f4dd84db4,True,2014-12-31
1,BF Karlsruhe Kaiserstr (1122),2014-12-31 23:00:40,1080,428fa91d6d741e1466b4bcd917dff4c2,True,2014-12-31


# 1-Create indexes table

In [4]:
def chunk_and_retrieve_indexes():
    chunks = pd.read_csv(MAIN_FILE,chunksize=1000000)
    
    for i,chunk in enumerate(chunks):
        print chunk.shape
        devices_ix = pd.DataFrame(chunk.device_id.unique())
        sites_ix = pd.DataFrame(chunk.site_name.unique())
        devices_ix.to_csv(DATA_PATH+"indexes/devices/devices_ix_"+str(i)+".csv")
        sites_ix.to_csv(DATA_PATH+"indexes/sites/sites_ix_.csv"+str(i)+".csv")

def concatenate_df(df,path):
    df1 = pd.read_csv(path,index_col=0)
   
    return pd.concat([df,df1]).drop_duplicates().reset_index(drop=True)

def concatenate_index_files():
    index_devices_files = pdh.get_files(DATA_PATH+"indexes/devices/")
    index_sites_files = pdh.get_files(DATA_PATH+"indexes/sites/")
    
    df_devices_ix = pd.DataFrame()
    df_sites_ix = pd.DataFrame()
    
    for index_device_file in index_devices_files:
        print index_device_file
        df_devices_ix = concatenate_df(df_devices_ix,DATA_PATH+"indexes/devices/"+index_device_file)
    df_devices_ix.columns=['id','device_mac']
    df_devices_ix.to_csv(DATA_PATH+"indexes/devices_ix.csv")
    
    for index_sites_file in index_sites_files:
        df_sites_ix = concatenate_df(df_sites_ix,DATA_PATH+"indexes/sites/"+index_sites_file)
    df_sites_ix.columns=['id','site_name']
    df_sites_ix.to_csv(DATA_PATH+"indexes/sites_ix.csv")

# 2-replace devices and sites name with ids


In [5]:
def merge_agg_with_device_and_sites_index():
    chunks=pd.read_csv(DATA_PATH+"nielsen.csv",chunksize=1000000)
    df_devices_idx = pd.read_csv(DATA_PATH+"indexes/devices_ix.csv")
    df_sites_index = pd.read_csv(DATA_PATH+"indexes/sites_ix.csv")
    df = pd.DataFrame()
    
    for i,chunk in enumerate(chunks):
        print chunk.shape, i
        chunk = pd.merge(chunk, df_devices_idx, left_on='device_id', right_on='device_mac',suffixes=('_agg', '_devicesidx'))
        chunk = chunk[['id','site_name','first_timeframe']]
        chunk = pd.merge(chunk, df_sites_index, on='site_name', suffixes=('_devices','_sites'))
        chunk=chunk[['id_devices', 'id_sites']]
        chunk=chunk.astype('int32')
        df = pd.concat([df,chunk], axis=0, ignore_index=True)
    print "merge over,starting saving to disk..."
    df.to_csv(DATA_PATH+'/nielsen_indexes.csv')
    print "saved to disk OK."

# 3-Aggregate per devices and site

In [6]:
def aggregate(chunk):
    
    chunk['count']=1
    groupy = chunk.groupby(['id_devices','id_sites']).count()
    groupy = groupy.reset_index()
    
    groupy =groupy.groupby('id_devices')['id_sites'].apply(lambda x: x.tolist())
    return pd.DataFrame(groupy)

def aggregate_per_devices_and_sites():
    chunks = pd.read_csv(DATA_PATH+'/nielsen_indexes.csv',chunksize=40000000,index_col=0)
    df =pd.DataFrame()
    
    for i,chunk in enumerate(chunks):
        df = pd.concat([df,aggregate(chunk)], axis=1, ignore_index=False)
        df = df.fillna(0)
        
        print df.shape,i
    df.to_csv(DATA_PATH+'/nielsen_indexes_pivot.csv')

# 4- Clean data


In [7]:
def combine_sites_id(df):
    cols = df.columns.tolist()
    df = df.replace('0', 0)
    df =df.replace(0,"")
    df = df.fillna("")
    df['sites_id']= df[cols].astype(str).sum(axis=1)
    df['sites_id_array']= df['sites_id'].apply(pdh.string_to_np_array)
    df['sites_count']= df['sites_id_array'].apply(lambda x: x.size)
    df = df[['sites_id_array','sites_count']]
    df.reset_index()
    return df
    
    
def clean_data():
    chunks = pd.read_csv(DATA_PATH+'nielsen_indexes_pivot.csv',chunksize=1000000,index_col=0)
    df = pd.DataFrame()
    
    for i,chunk in enumerate(chunks):
        df = pd.concat([df,combine_sites_id(chunk)],axis=0)
        print df.shape,i
    print "Combining done, saving to file..."
    df.to_csv(DATA_PATH+"nielsen_indexes_sites_per_devices.csv")

# 5- Filter data on devices that appeared on more than one site
once a list of devices/sites/number of sites detected has been created, we can easily filter on devices with multiple site appearances to reduce the dataset of 67%, to a dataset of around 6 millions devices :

In [3]:
def create_mask_on_devices_that_appears_on_multiple_sites():
    df = pd.read_csv(DATA_PATH+"/nielsen_indexes_sites_per_devices.csv", index_col=0)
    mask = pd.DataFrame(df[df.sites_count>1].index.values)
    mask.columns=['device_id']
    
    df_devices_indexes = pd.read_csv(DATA_PATH+"indexes/devices_ix.csv")
    df = pd.merge(mask,df_devices_indexes,left_on="device_id",right_on="id")
    df=df[['device_id','device_mac']]
    df.to_csv(DATA_PATH+"indexes/devices_ix_multiple_sites.csv")

# 6- Merge devices with multiple sites with raw data.

In [9]:
def merge_data(chunk,mask,sites_id):
    chunk = chunk.reset_index()
    chunk['date']=chunk['first_timeframe'].dt.date

    df1 = pd.merge(chunk,mask,left_on='device_id',right_on='device_mac')[['device_id_y','site_name','date']]
    df1 = pd.merge(df1,sites_id,on='site_name')[['device_id_y','id','date']]
    df1.columns = ['device_id','site_id','date']
    return df1

def reduce_data_on_mask():
    devices_id_chunks = pd.read_csv(DATA_PATH+"indexes/devices_ix_multiple_sites.csv", chunksize=500000,index_col=0)
    sites_id = pd.read_csv(DATA_PATH+"indexes/sites_ix.csv",index_col=0)
    sites_id = sites_id.reset_index()  
    
    for i,devices_id_chunk in enumerate(devices_id_chunks):
        data_chunks = pd.read_csv(DATA_PATH+'nielsen.csv',parse_dates=['first_timeframe'],chunksize=1000000,index_col=0)
        print "Merging "+str(i)+" Starts.------------------------------"
        df = pd.DataFrame()
        
        for j,data_chunk in enumerate(data_chunks):
            df = pd.concat([df,merge_data(data_chunk,devices_id_chunk,sites_id)],axis=0)
            print df.shape,j
        print "Finished merging, saving to file..."
        df.to_csv(DATA_PATH+"nielsen_data_multiple_sites_"+str(i)+".csv")

# 7-Group filtered raw data per device-date-site

In [10]:
def final_count(file_name):
    df = pd.read_csv(DATA_PATH+"data_multiples_sites_only_with_indexes/"+file_name,index_col=0)
    result = df.groupby(['device_id','date'])['site_id'].apply(lambda x: (np.unique(x.values).size))
    final = pd.DataFrame(result)
    final.columns=[['count_sites']]
    file_ix = re.findall(r'\d+',file_name)[0]
    final.to_csv(DATA_PATH+"groupby_device_date_site/groupby_device_date_site"+str(file_ix)+".csv")
    
def groupby_device_date_site():
    files = pdh.get_files(DATA_PATH+"data_multiples_sites_only_with_indexes/")
    
    for filo in files:
        print "grouping :"+ filo
        final_count(filo)
    
    groupby_files = pdh.get_files(DATA_PATH+"groupby_device_date_site/")
    df = pd.DataFrame()
    for groupbyfilo in groupby_files:
        df1 = pd.read_csv(DATA_PATH+"groupby_device_date_site/"+groupbyfilo,index_col=0)
        df = pd.concat([df,df1],axis=0)
    df.to_csv(DATA_PATH+"groupby_device_date_site_final.csv")



# 8- Some statistics

In [11]:
def get_dataset_stats():
    chunks = pd.read_csv(DATA_PATH+"nielsen.csv",parse_dates=['first_timeframe'],chunksize=1000000)
    stats =  {}
    min_date = date.today()
    max_date = date(2000,1,1)
    rows =0
    for c in chunks:
        rows+=c.shape[0]
        c.first_timeframe = c.first_timeframe.dt.date
        min_c,max_c = c.first_timeframe.min(), c.first_timeframe.max()
        if min_c < min_date:
            min_date = min_c
        if max_c > max_date:
            max_date = max_c
        print "{:,}".format(rows),min_date,max_date
    stats = {'rows':rows,'min_date':min_date,'max_date':max_date}            
    return stats


def get_number_of_devices():
    df = pd.read_csv(DATA_PATH+"indexes/devices_ix.csv",index_col=0)
    return df.shape[0]

def get_number_of_sites():
    df = pd.read_csv(DATA_PATH+"indexes/sites_ix.csv",index_col=0)
    return df.shape[0]

def get_numbers_records_multi_sites_devices():
    rows = 0
    files = pdh.get_files(DATA_PATH+"data_multiples_sites_only_with_indexes/")
    for filo in files:
        df1 = pd.read_csv(DATA_PATH+"data_multiples_sites_only_with_indexes/"+filo,index_col=0) 
        rows+=df1.shape[0]
    return rows

## <pre>
<center>

| Dataset total records |  167,944,229 |
| :- | :-:
|   Number of devices  | 21,010,205|
|   Number of sites  | 53|
|   Number of devices on more than 2 sites (multi-sites devices) | 1,994,526|
|   Records for multi-sites devices  | 47,965,794|
|   Date range of records  | 2014-12-31 to 2015-12-31|

</center>
</pre>

In [12]:
def display_visits_repartition():
    df = pd.read_csv(DATA_PATH+"groupby_device_date_site_final.csv",index_col=0)
    df.columns=[['date','count_sites']]
    groupy = df.reset_index()
    groupy['date']= pd.to_datetime(groupy.date)
    groupy['weekday']= groupy['date'].dt.dayofweek
    groupy['day_count']= (groupy.date-date(2014,12,31)) / np.timedelta64(1, 'D')
    
    return groupy

def scatter_visits(groupy):
    groupy_more_than_1_site = groupy[groupy.count_sites>1]
    plt.scatter(groupy_more_than_1_site.day_count,groupy_more_than_1_site.count_sites,alpha=0.7, c='r')
    plt.xlabel("day from 1st Jan 2015", fontsize=12)
    plt.ylabel("Number of sites visited per day",fontsize=12)
    plt.title('Visited sites per device')
    visited_sites_per_device_per_year = pd.DataFrame(groupy.groupby('device_id')['count_sites'].mean())
    visited_sites_per_device_per_year = visited_sites_per_device_per_year.sort_values('count_sites',ascending=False)
    visited_sites_per_device_per_year= visited_sites_per_device_per_year.reset_index()
    groupy_visits = visited_sites_per_device_per_year.groupby(['count_sites']).count()
    groupy_visits= groupy_visits.sort_values(by="device_id",ascending=False)
    print groupy_visits.device_id.sum(axis=0)
    groupy_visits['cumsumo']=groupy_visits.device_id/groupy_visits.device_id.sum(axis=0)*100
    return groupy_visits[groupy_visits.device_id>1]

In [13]:
def draw_visits(groupy):
    devices_per_day_weekday = groupy[groupy.weekday<6]
    devices_per_day_weekend = groupy[groupy.weekday==6]
    devices_per_day_weekday = devices_per_day_weekday.groupby('date')['count_sites'].count()
    devices_per_day_weekend = devices_per_day_weekend.groupby('date')['count_sites'].count()
    devices_per_day_weekday = devices_per_day_weekday.sort_index(ascending=True)
    devices_per_day_weekend = devices_per_day_weekend.sort_index(ascending=True)
    devices_per_day_weekday.plot(alpha=0.7,c='g',title="Number of multi-sites devices recorded per day")
    devices_per_day_weekend.plot(alpha=0.7,c='r',legend='Sunday')

In [49]:
def search_device_mac(device_id_list):
    df = pd.read_csv(DATA_PATH+"/indexes/devices_ix_multiple_sites.csv")
    return df[df.device_id.isin(device_id_list)]

def search_device_data(mac_list):  
    chunks = pd.read_csv(DATA_PATH+"/nielsen.csv",chunksize = 1000000)
    df = pd.DataFrame()
    for i,c in enumerate(chunks):
        df1 = c[c.device_id.isin(mac_list)]
        print df1.shape,i
        df = pd.concat([df,df1],axis=0)
    df.to_csv(DATA_PATH+"search_result.csv")

In [15]:
counts = display_visits_repartition()

In [16]:
counts.head()

,device_id,date,count_sites,weekday,day_count
0,2224071,2015-01-05,1,0,5.0
1,2224071,2015-01-13,1,1,13.0
2,2224071,2015-01-20,1,1,20.0
3,2224071,2015-01-26,1,0,26.0
4,2224071,2015-01-29,1,3,29.0


In [67]:
visits =counts.groupby('device_id')['count_sites'].mean()
visits = pd.DataFrame(visits)
visits= visits.sort_values(by='count_sites',ascending=True)
devices = visits[visits.count_sites==1].index.tolist()[:10]
devices+= visits[visits.count_sites>4].index.tolist()[:10]
visits.head()


,count_sites
device_id,
24,1.0
4749151,1.0
4749147,1.0
4749136,1.0
4749097,1.0


In [24]:
device_mac_list = search_device_mac(devices).device_mac.tolist()

In [59]:
device_mac_list

['76337b2689839e68b22235bdbb00ae5f',
 '71085a86d20b64c43ec3b9781ef70cfc',
 'fd5ab440ac21cc7726883d844b119861',
 '96b40b45252bdcf12ffc90a1c5995f96',
 '1d378b6df55bcc9dea58b6728e7e8285',
 'd412d012c3961a85258a970f1e19c82b',
 '873dc43cc36d8b66dd295625ec9c2b56',
 '200f14d66a5e433d1de35ca38b15224d',
 '86e5f3f97ae1c2f87d6dcfd066b907c1',
 'a470147fa8e70547a380b6b3404bea37',
 'cc3d9179915f97e328bbbe3959c66fa9',
 '711754e35c5dc013bd805b8a668a781f',
 '3bd28e9d0ebf7bb8475e3aa430b03298',
 '8afc3408ca592ee9d3261533d05706d6',
 'd3c740597cf2d59b14de69c6a1a17c68',
 '9f45f76554d3a330e3c32d8e458e1fde',
 'aeb2fa0bb47631e84a61a3b3471e9722']

In [39]:
type(device_mac_list)

list

In [41]:
df = pd.read_csv(DATA_PATH+"/nielsen.csv",nrows=10000)

In [50]:
search_device_data(device_mac_list)

(256, 5) 0
(210, 5) 1
(269, 5) 2
(297, 5) 3
(264, 5) 4
(304, 5) 5
(287, 5) 6
(270, 5) 7
(279, 5) 8
(263, 5) 9
(269, 5) 10
(362, 5) 11
(253, 5) 12
(256, 5) 13
(255, 5) 14
(176, 5) 15
(221, 5) 16
(196, 5) 17
(318, 5) 18
(189, 5) 19
(173, 5) 20
(275, 5) 21
(798, 5) 22
(367, 5) 23
(334, 5) 24
(310, 5) 25
(260, 5) 26
(221, 5) 27
(220, 5) 28
(189, 5) 29
(195, 5) 30
(144, 5) 31
(381, 5) 32
(305, 5) 33
(231, 5) 34
(206, 5) 35
(219, 5) 36
(175, 5) 37
(164, 5) 38
(190, 5) 39
(238, 5) 40
(367, 5) 41
(178, 5) 42
(244, 5) 43
(457, 5) 44
(124, 5) 45
(194, 5) 46
(271, 5) 47
(280, 5) 48
(306, 5) 49
(308, 5) 50
(244, 5) 51
(222, 5) 52
(201, 5) 53
(226, 5) 54
(209, 5) 55
(131, 5) 56
(516, 5) 57
(257, 5) 58
(323, 5) 59
(226, 5) 60
(227, 5) 61
(206, 5) 62
(170, 5) 63
(144, 5) 64
(185, 5) 65
(224, 5) 66
(158, 5) 67
(151, 5) 68
(219, 5) 69
(216, 5) 70
(270, 5) 71
(229, 5) 72
(220, 5) 73
(157, 5) 74
(153, 5) 75
(194, 5) 76
(148, 5) 77
(110, 5) 78
(148, 5) 79
(209, 5) 80
(177, 5) 81
(334, 5) 82
(144, 5) 83
(1

/home/ramon/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(248, 5) 167


In [51]:
df = pd.read_csv(DATA_PATH+"search_result.csv")


In [65]:
groupy = df[df.device_id=='aeb2fa0bb47631e84a61a3b3471e9722']

In [66]:
groupy

,Unnamed: 0,site_name,first_timeframe,dwell_time_s,device_id,visitor
1661,141577,BF Berlin Brunnenstr (2123),2015-03-31 12:46:50+00:00,220,aeb2fa0bb47631e84a61a3b3471e9722,False
6226,696089,BF Berlin Friedrichstr (1113),2015-04-16 15:54:00+00:00,5,aeb2fa0bb47631e84a61a3b3471e9722,False
27763,186582,BF Kassel Obere Königsstr (1132),2015-08-22 10:47:30+00:00,15,aeb2fa0bb47631e84a61a3b3471e9722,False


In [62]:
groupy = groupy.groupby(['site_name'])['first_timeframe'].count()

In [63]:
 
groupy

site_name
BF Berlin Brunnenstr (2123)         1
BF Berlin Friedrichstr (1113)       1
BF Kassel Obere Königsstr (1132)    1
Name: first_timeframe, dtype: int64